### 2022 Shopify Data Science Intern Challenge
##### Name: Elie Saliba
---

### Question 1

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe.\
We want to do some analysis of the average order value (AOV).\
When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13\.\
Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

a. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.\
b. What metric would you report for this dataset?\
c. What is its value?

---


##### Tiny sidenote before getting started...
I explored the dataset fully and noticed that the 'created_at' dates did not correspond to the chronologically incrementing order_ids.\
Normally this would warrant investigation from the Data Engineering team, but for the sake of this challenge, I am moving forward\
under the assumption that this does not matter.

An example of what I mean is:\
order_id '1104' created at '2017-03-24 4:00:00'\
somehow preceeds\
order_id '1363'	which is actually created before at '2017-03-15 4:00:00'

### Data Exploration

In [1]:
# Import required libraries

import numpy as np
import pandas as pd

In [2]:
# Assigning the csv to a pandas dataframe object
df = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")

# Showing the first 5 rows of the dataframe
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [3]:
# Checking for null values in the df
df.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

In [4]:
# Let's take a quick glance at some metrics from the dataframe
# I restricted the columns to 'order_amount' and 'total_items' since the 'id' columns are nominal despite being numeric
df[['order_amount', 'total_items']].describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


#### At A Glance
---
From the above output, we can see that there is at least one outlier in both the 'order_amount' and the 'total_items' columns.\
We can draw this conclusion because there is clearly a very large difference between the 75th percentile values and the maximum values:

|            | 75th percentile | Maximum  |
|:-----------|----------------:|---------:|
|order_amount| 309             | 704,000  |
|total_items | 3               | 2,000    |

\
Knowing that sneakers are typically affordable items that customers usually buy 1-2 of at a time we can infer that one of the\
following scenarios is the cause of these outliers:

* The outliers are typos/errors
* The outliers are a result of niches within the sneaker market

Perhaps the order_amount outliers correspond to luxury sneaker shops, or maybe the total_items and order_amount outliers\
are linked due to a bulk order from a reseller.

We will be able to make a more informed judgement on the data as we explore it further!

In [5]:
# Creating a sorted list of the unique values in the 'order_amount' column
order_amt_sorted = sorted((df['order_amount'].unique()))

# This will allow us to look at the 10 highest values in that list
order_amt_sorted[-10:]

[1064, 1086, 1408, 1760, 25725, 51450, 77175, 102900, 154350, 704000]

#### Note:
---
We can see that it is unlikely that all of these outliers are typos.\
They are likely results of bulk purchases and the odd designer sneaker store.

In [6]:
# Lets see what we can glean from looking only at records where the order_amount is 704,000

df[df['order_amount'] == 704000]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00


#### Note:
---
We can see that shop_id '42' is solely responsible for the 704,000 order amounts,\
and all of those orders happen to be bulk orders of 2,000 items made by the same user_id '607'\
User 607 could be a reseller or this could be a flag for further investigation as possible fraud.\
Either way, this would prompt further probing but is outside of the scope of this question.

In [7]:
# Let's see what we can find looking at the top 10 most expensive order_amounts
# but while limiting total_items to only 1 item

# Uncomment to view table
# df[(df['order_amount'] >= 1064) & (df['total_items'] == 1)]

In [8]:
list(df['shop_id'][(df['order_amount'] >= 390) & (df['total_items'] == 1)].unique())

[78]

#### Note:
---
We can see here that the only shop that has order amounts above the 75th percentile is shop_id '78'.\
But we can gain a further understanding of the order amounts if we dig just a little deeper.

Let's get the order amount of every store that has sold only 1 item in a transaction,\
this will give us the price of the sneaker being sold by that shop!

In [9]:
# First, let's check that all the shops do in fact have a sale of just 1 item per transaction at some point
df[df['total_items'] == 1].nunique()

order_id          1830
shop_id            100
user_id            299
order_amount        58
total_items          1
payment_method       3
created_at        1827
dtype: int64

Great! We can see that there are 100 unique shop_id's here.\
This lines up with the information from the question prompt telling us that there are\
exactly 100 sneaker stores on Shopify in this example.

In [10]:
# Now let's look at the list of the unique order amounts from sales that had a total_item count of 1
# the print function is just to quickly put the output into a more compact format
print(sorted(df['order_amount'][df['total_items'] == 1].unique()))

[90, 94, 101, 111, 112, 114, 116, 117, 118, 122, 127, 128, 129, 130, 131, 132, 133, 134, 136, 138, 140, 142, 144, 145, 146, 147, 148, 149, 153, 154, 155, 156, 158, 160, 161, 162, 163, 164, 165, 166, 168, 169, 171, 172, 173, 176, 177, 178, 181, 184, 187, 190, 193, 195, 196, 201, 352, 25725]


#### Note:
---
It is apparent now that all order amounts greater than 352 ( with the exception of the far outlier 25,725 )\
are simply orders where more than 1 sneaker was sold.

Let's use the Interquartile range method to determine outliers next!\
This dictates that outliers fall more than 1.5 x IQR below Q1 or above Q3.

In [11]:
# Setting the our list of unique order amounts to an object for readability
data = sorted(df['order_amount'][df['total_items'] == 1].unique())

# Finding and assigning our quartiles using numpy's percentile function
q1, q3 = np.percentile(data, [25, 75])
# The equation for interquartile range (iqr)
iqr = q3 - q1

# Displaying the values we found
iqr, q1, q3

(40.5, 131.25, 171.75)

In [12]:
# For larger datasets we should use mapping or a filter to return a list of non-outliers
# but this set is small enough to simply do the calculation manually

outlier_tester = 1.5 * iqr
print(90 >= q1 - outlier_tester)
print(25725 <= q3 + outlier_tester)
print(352 <= q3 + outlier_tester)
print(201 <= q3 + outlier_tester)

True
False
False
True


In [13]:
df[(df['order_amount'] == 352) & (df['total_items'] == 1)]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
40,41,42,793,352,1,credit_card,2017-03-24 14:15:41
308,309,42,770,352,1,credit_card,2017-03-11 18:14:39
834,835,42,792,352,1,cash,2017-03-25 21:31:25
979,980,42,744,352,1,debit,2017-03-12 13:09:04
1512,1513,42,946,352,1,debit,2017-03-24 13:35:04
1929,1930,42,770,352,1,credit_card,2017-03-17 8:11:13
2018,2019,42,739,352,1,debit,2017-03-01 12:42:26
2053,2054,42,951,352,1,debit,2017-03-19 11:49:12
3651,3652,42,830,352,1,credit_card,2017-03-24 22:26:58
3697,3698,42,839,352,1,debit,2017-03-12 2:45:09


#### Note:
---
Using this, we can see that we have no outliers on the lower bounds of the 'order_amount' data,\
but we do have 2 outliers on the upper bounds:
* 352 ( 'shop_id' = 42 )
* 25,725 ( 'shop_id' = 78 )

We would have removed records of shop_id '42' even if the price per sneaker wasn't an outlier on its own\
as the records of $704,000 per order are clear outliers and warrant further exploration.

In [14]:
# Let's make a df that excludes the outliers
df_clean = df[(df['shop_id'] != 42) & (df['shop_id'] != 78)]

In [15]:
# Here we'll check the remaining records in the df
df_clean.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4903.000000,4903.000000,4903.000000,4903.000000,4903.000000
mean,2499.584540,49.900877,849.858862,300.155823,1.995717
std,1444.221163,29.154367,86.887947,155.941112,0.982602
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1246.500000,24.000000,776.000000,163.000000,1.000000
50%,2499.000000,50.000000,850.000000,284.000000,2.000000
75%,3750.500000,74.000000,925.000000,386.500000,3.000000
max,5000.000000,100.000000,999.000000,1086.000000,8.000000


In [16]:
# Double checking that the correct amount of records were removed

# df[df['shop_id'] == 42].count()
# df[df['shop_id'] == 78].count()

We can see above that 'df_clean' has had 97 records removed, which corresponds to the amount of\
records due to the outliers in the original df.

In [17]:
# Here we will create an object with the shop_id's from the cleaned dataframe
# This list will be used to iterate through the dataframe for our calculations
shop_ids_clean = sorted(list(df_clean['shop_id'].unique()))

In [18]:
# Let's group the records by shop_id to make it easier to calculate the metrics we want to use
df_group = df_clean.groupby(by = 'shop_id')

In [19]:
# Let's calculate the AOV for every store

# Initializing an empty list to store each store's AOV
aov_per_store = []

# This for loop iterates through the list of shop_id's we just created
for i in shop_ids_clean:
    aov_per_store.append(((df_group.get_group(i)['order_amount']).sum()) / (df_group.get_group(i)['order_amount'].count()))

In [ ]:
# Let's have a look at the output
aov_per_store

In [21]:
# Now we will calculate a 'weighted' AOV for all the stores by taking the average of all of their AOVs!
sum(aov_per_store) / len(aov_per_store)

299.68239912615485

In [22]:
# Let's also look at the median of the order_amounts for the entire dataset
df['order_amount'].median()

284.0

## Time to Answer!
---

#### a. Think about what could be going wrong with our calculation. Think about a better way  to  evaluate this data.

    For starters, we can immediately see that the naive AOV calculation of 3145.13 dollars would only work in
    the scenario where each store only sold 1 item per order.

    Even then, this naive calculation does not account for any outliers and so will be heavily skewed
    and consequently misrepresenting the data!

    There are multiple ways to tackle this problem depending on how we aim to ultimately use this data.
    A better way to evaluate this data using this calculation as an approach would be to instead take
    the AOV for each store individually, and then finding the average of all of the calculated AOV's to
    provide us with a weighted AOV that accounts for all the stores (with the outliers removed).
    
    Since no one metric could possibly provide the whole story for this dataset (especially considering
    the fact that we are cleaning the data and removing outliers), we could also include the median of
    all order values to give a picture of the central tendency of the data while minimizing the effect
    of the outliers. This can be done on the df pre-cleaning.

#### b. What metric would you report for this dataset?

    A weighted AOV of all stores that is calculated by taking the AOV of each store, summing them up and then
    dividing by the total number of stores.
    
    I would also report the median of all order amounts.
   

#### c. What is its value?

    The value of the 'weighted' AOV of all stores (which we found to be 299.68 dollars) is that it accounts for different stores
    having differently priced items and different numbers of sales. As a result of these different prices and sales numbers,
    the different stores do not equally contribute to the sum of the 'order_amount' column. Without accounting for the different
    weights of each shop, we would end up with the problematic naive AOV of 3145.13 which was present in the question.
    Instead, the weighted AOV provides us with a more accurate picture of what we can expect from the stores in this dataset,
    with the exception of the 2 outliers.

    The value of the median of order amounts (calculated to be 284 dollars) lies in its robustness against outliers and helping us
    understand whereabouts the "middle" of our data is.